In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# For BART, T5, Pegasus
#!pip3 install -q transformers==3.5.0
!pip3 install -q transformers==4.15.0

!pip3 install -q torch==1.7.0
!pip3 install -q sentencepiece

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 61 kB 498 kB/s 
     |████████████████████████████████| 596 kB 77.8 MB/s 
     |████████████████████████████████| 895 kB 82.3 MB/s 
     |████████████████████████████████| 3.3 MB 61.2 MB/s 
     |████████████████████████████████| 776.7 MB 3.7 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
     |████████████████████████████████| 1.2 MB 4.4 MB/s 


In [ ]:
# Checking out the GPU we have access to
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip3 install -q scispacy
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

!python3 -m spacy download en

     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 71 kB 4.3 MB/s 
     |████████████████████████████████| 13.5 MB 20.5 MB/s 
     |████████████████████████████████| 6.3 MB 27.2 MB/s 
     |████████████████████████████████| 188 kB 82.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 451 kB 85.8 MB/s 
     |████████████████████████████████| 628 kB 81.2 MB/s 
     |████████████████████████████████| 10.1 MB 82.8 MB/s 
     |████████████████████████████████| 33.1 MB 1.4 MB/s 
     |████████████████████████████████| 125.1 MB 44 kB/s 
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7 MB 2.0 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5


In [ ]:
!pip3 install -q jsonlines

In [ ]:
import os
import numpy as np
import pandas as pd
import json
import jsonlines
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import (T5Tokenizer, T5ForConditionalGeneration, 
                          BartTokenizer, BartForConditionalGeneration,
                          PegasusTokenizer, PegasusForConditionalGeneration,
                          LEDTokenizer, LEDForConditionalGeneration,
                          BigBirdPegasusForConditionalGeneration, AutoTokenizer)


In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(device)

cpu


In [ ]:
with open("input_args.json") as fp:
  input_args = json.load(fp)
fp.close()

In [ ]:
input_args

{'DATA_PATH': './',
 'EPOCHS': 3,
 'LEARNING_RATE': 0.0001,
 'MAX_LEN': 1500,
 'SUMMARY_LEN': 210,
 'TRAIN_BATCH_SIZE': 1,
 'VALID_BATCH_SIZE': 1,
 'filename': 'pubmed_w_named_entities_25k.jsonl',
 'model': 'bigbird',
 'model_type': 'vanilla',
 'train_size': 0.8,
 'w_named_entity': False}

### Read and parse the pubmed dataset

In [ ]:
def _read_parse_data(filepath, w_named_entity=False):
  data, final_data = [], []
  with open(filepath) as f:
    for line in f:
      data.append(json.loads(line))
  f.close()

  if w_named_entity:
    for article in data:
      article_abstract = "".join(article['article_abstract']).replace('<S>', '').replace('</S>', '')
      article_text = "".join(article['article_text'])
      article_text_named_entities = " ".join(article['article_text_named_entities'].split(' | '))
      final_data.append([article_text, article_abstract, article_text_named_entities])

    df = pd.DataFrame(final_data, columns=['article_text', 'article_abstract', 'article_text_named_entities'])
    df["article_text_w_named_entites"] = df["article_text_named_entities"] + "[SEP]" + df["article_text"]
    df.article_text = 'summarize: ' + df.article_text_w_named_entites
    df = df.drop(columns=['article_text_named_entities', 'article_text_w_named_entites'])

  else:
    for article in data:
      article_abstract = "".join(article['article_abstract']).replace('<S>', '').replace('</S>', '')
      article_text = "".join(article['article_text'])
      final_data.append([article_text, article_abstract])

    df = pd.DataFrame(final_data, columns=['article_text', 'article_abstract'])
    df.article_text = 'summarize: ' + df.article_text

  return df

In [ ]:
def main():
  DATA_PATH = './'
  filename = input_args['filename']
  filepath = f"{DATA_PATH}/{filename}"

  w_named_entity = input_args['w_named_entity']

  # call to the method above
  df = _read_parse_data(filepath, w_named_entity)

  return df


In [ ]:
if __name__ == "__main__":
  df = main()

In [ ]:
df.head()

,article_text,article_abstract
0,summarize: a recent systematic analysis showed...,background : the present study was carried ou...
1,summarize: it occurs in more than 50% of patie...,backgroundanemia in patients with cancer who ...
2,"summarize: tardive dystonia ( td ) , a rarer s...",tardive dystonia ( td ) is a serious side eff...
3,summarize: lepidoptera include agricultural pe...,many lepidopteran insects are agricultural pe...
4,summarize: syncope is caused by transient diff...,we present an unusual case of recurrent cough...


## Select which tokenizer and model to fine-tune

In [ ]:
# Defining some key variables that will be used later on in the training

MAX_LEN = input_args['MAX_LEN']
SUMMARY_LEN = input_args['SUMMARY_LEN']
TRAIN_BATCH_SIZE = input_args['TRAIN_BATCH_SIZE']
VALID_BATCH_SIZE = input_args['VALID_BATCH_SIZE']
EPOCHS = input_args['EPOCHS']
LEARNING_RATE = input_args['LEARNING_RATE']

In [ ]:
# Sections of config
def _selectModel(modelName):
  if modelName.lower() == "t5":
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
  elif modelName.lower() == 'bart':
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
  elif modelName.lower() == 'pegasus':
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
    model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
  elif modelName.lower() == 'longformer':
    tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
    model = LEDForConditionalGeneration.from_pretrained('allenai/led-base-16384')
  elif modelName.lower() == "bigbird":
    tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")
    # by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
    model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed")

  
  model = model.to(device)

  return model, tokenizer

In [ ]:
def main():
  #modelName = "BART"
  modelName = input_args["model"]
  model, tokenizer = _selectModel(modelName)

  return model, tokenizer

In [ ]:
if __name__ == "__main__":
  model, tokenizer = main()

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

### Instantiate an optimizer

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage 
# for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.article_abstract = self.data.article_abstract
        self.article_text = self.data.article_text

    def __len__(self):
        return len(self.article_abstract)

    def __getitem__(self, index):
        article_text = str(self.article_text[index])
        article_text = ' '.join(article_text.split())

        article_abstract = str(self.article_abstract[index])
        article_abstract = ' '.join(article_abstract.split())

        source = self.tokenizer.batch_encode_plus([article_text], max_length= self.source_len, pad_to_max_length=True,
                                                  truncation=True, return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([article_abstract], max_length= self.summ_len, pad_to_max_length=True,
                                                  truncation=True, return_tensors='pt')


        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = input_args['train_size']
train_dataset=df.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

FULL Dataset: (4888, 2)
TRAIN Dataset: (3910, 2)
TEST Dataset: (978, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
def train(epoch):
  model.train()
  for _, data in enumerate(training_loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

    loss = outputs[0]
    
    if _%500==0:
        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def _get_config_file(model_path, model_name):
    # Name of the file for storing hyperparameter details
    return os.path.join(model_path, model_name + ".config")

def _get_model_file(model_path, model_name):
    # Name of the file for storing network parameters
    return os.path.join(model_path, model_name + ".tar")

def load_model(model_path, model_name, net=None):
    """
    Loads a saved model from disk.

    Inputs:
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
        net - (Optional) If given, the state dict is loaded into this model. Otherwise, a new model is created.
    """
    config_file, model_file = _get_config_file(model_path, model_name), _get_model_file(model_path, model_name)
    assert os.path.isfile(config_file), f"Could not find the config file \"{config_file}\". Are you sure this is the correct path and you have your model config stored here?"
    assert os.path.isfile(model_file), f"Could not find the model file \"{model_file}\". Are you sure this is the correct path and you have your model stored here?"
    with open(config_file, "r") as f:
        config_dict = json.load(f)
    if net is None:
        act_fn_name = config_dict["act_fn"].pop("name").lower()
        act_fn = act_fn_by_name[act_fn_name](**config_dict.pop("act_fn"))
        net = BaseNetwork(act_fn=act_fn, **config_dict)
    net.load_state_dict(torch.load(model_file, map_location=device))
    return net

def save_model(model, model_path, model_name):
    """
    Given a model, we save the state_dict and hyperparameters.

    Inputs:
        model - Network object to save parameters from
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
    """
    config_dict = model.config
    os.makedirs(model_path, exist_ok=True)
    config_file, model_file = _get_config_file(model_path, model_name), _get_model_file(model_path, model_name)
    
    with open(config_file, "w") as f:
      json.dump(config_dict, f)
    f.close()

    torch.save(model.state_dict(), model_file)

In [ ]:
!pip3 install -q torch==1.7.0
!pip3 install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import torch
import transformers
import os

### Call to the train method above

In [ ]:
def main(model):
  for epoch in range(EPOCHS):
    train(epoch)

  model_name = input_args['model']
  model_type = input_args['model_type']   # changes with type (i.e., vanilla, entity, entity-fact)

  model_path = f"pubmed-FINAL-CHECKPTS/{model_name}"
  os.makedirs(model_path, exist_ok=True)

  # Save entire model to a path
  final_model_path = f"{model_path}/{model_name}-{model_type}.pt"
  torch.save(model, final_model_path)

  ###############################################################################################

  # Save checkpoint to the file system
  MODEL_DIRECTORY_PATH = f"pubmed-FINAL-CHECKPTS/{model_name}"
  model_path = f"{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}.pt"
  model = torch.load(model_path)

  os.makedirs(f'{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}-checkpoints')

  model.save_pretrained(f'{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}-checkpoints/')


In [ ]:
if __name__ == "__main__":
  main(model)

Epoch: 0, Loss:  7.355123519897461
Epoch: 0, Loss:  3.2026078701019287
Epoch: 0, Loss:  1.3980499505996704
Epoch: 0, Loss:  1.1156871318817139
Epoch: 0, Loss:  1.2584503889083862
Epoch: 0, Loss:  1.3448430299758911
Epoch: 0, Loss:  1.1389212608337402
Epoch: 0, Loss:  1.1897244453430176
Epoch: 1, Loss:  1.3530696630477905
Epoch: 1, Loss:  0.8287136554718018


## Saving the model as a checkpoint and loading back to generate summaries

In [ ]:
!pip3 install -q torch==1.7.0
!pip3 install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import torch
import transformers
import os

### Saving as checkpoints

In [ ]:
#model_name = input_args['model']
#model_type = input_args['model_type']

model_name = "longformer"
model_type = "vanilla"

# model path
MODEL_DIRECTORY_PATH = f"pubmed-FINAL-CHECKPTS/{model_name}"

model_path = f"{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}.pt"
model = torch.load(model_path)

os.makedirs(f'{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}-checkpoints')

model.save_pretrained(f'{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}-checkpoints/')

### Loading entire model from the file

In [ ]:
from torchsummary import summary

model_name = "T5"
model_type = "vanilla"

# model path
MODEL_DIRECTORY_PATH = f"pubmed-FINAL-CHECKPTS/{model_name}"
model_path = f"{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}.pt"

#model = model.load_state_dict(torch.load(model_path))
model = torch.load(model_path)

In [ ]:
#model

In [ ]:
#model.state_dict()

### Experiment with generating summaries. Note that during inference, we shall experiment with the different input configurations

In [ ]:
import spacy
import scispacy
import pandas as pd
import numpy as np
import pickle as pk
from pprint import pprint
import os

from spacy import displacy
import en_core_sci_sm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker

In [ ]:
def _loadModel(model_name, model_path):
  if model_name == 'BART':
    model = BartForConditionalGeneration.from_pretrained(model_path)
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
  elif model_name == 'T5':
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
  elif model_name == "Pegasus":
    model = PegasusForConditionalGeneration.from_pretrained(model_path)
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

  return model, tokenizer

In [ ]:
def _generate_abstractive_summary(input_text, model, tokenizer, MAX_LEN=250):
  inputs = tokenizer(input_text, truncation=True, padding='longest',return_tensors='pt').to(device)
  # Get the highest-scoring beam as the abstractive summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=70, max_length=MAX_LEN, 
                               early_stopping=True, num_return_sequences=1)
  
  tgt_text = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
  abstractive_summary = tgt_text[0]

  return abstractive_summary


In [ ]:
def _get_named_entities(input_text):
  nlp = spacy.load("en_ner_bc5cdr_md")
  doc = nlp(input_text)
  entities = []
  for ent in doc.ents:
    entities.append(ent.text)
  str_entities = " | ".join(entities)   # a string representation of list of entities with the pipe symbol as a separator
  
  return str_entities

In [ ]:
def main():
  lst_modelName = ["Pegasus"]
  lst_model_type = ["w_named_entities"]

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      inference_type = "w_named_entities"   # determines what to pass during model inference time (testing time)

      input_fileName = "pubmed_w_named_entities_25k.jsonl"
      output_filename = f"pubmed-summaries-{model_type}.jsonl"

      # model path
      MODEL_DIRECTORY_PATH = f"pubmed-FINAL-CHECKPTS/{modelName}" 
      model_path = f'{MODEL_DIRECTORY_PATH}/{modelName}-{model_type}-checkpoints/'   # this is checkpoint path in other words

      model, tokenizer = _loadModel(modelName, model_path)
      model = model.to(device)


      SUMMARY_OUTPUT_PATH = f"pubmed-FINAL-SUMMARIES_w_named_entites_at_inference/{modelName}"
      os.makedirs(SUMMARY_OUTPUT_PATH, exist_ok=True)
      
      with open(f"{input_fileName}") as f:
        
        for idx, line in enumerate(f):
          if idx > 2071:
            if idx % 2000 == 0:
              print("Iteration: ", idx)
            dict_data = json.loads(line)
            
            if inference_type == "w_named_entities":
              input_text = dict_data['article_text_named_entities'] + "[SEP]" + dict_data['article_text']
            else:
              input_text = dict_data['article_text']  # this is the vanilla input configuration at inference time (i.e., without named entities)
            summary = _generate_abstractive_summary(input_text, model, tokenizer)

            summary_named_entities = _get_named_entities(summary)  # named entities for the generated summary

            dict_1 = {"article_text" : dict_data['article_text'],
                      "article_abstract" : dict_data['article_abstract'],
                      "article_text_named_entities" : dict_data['article_text_named_entities'],
                      "article_abstract_named_entities" : dict_data['article_abstract_named_entities'],
                      "abstractive_summary" : summary,
                      "abstractive_summary_named_entities" : summary_named_entities
                      }
                
            with jsonlines.open(f"{SUMMARY_OUTPUT_PATH}/{output_filename}", "a") as writer:
              writer.write(dict_1)
            writer.close()
          
      f.close()

In [ ]:
if __name__ == "__main__":
  main()

Iteration:  4000
